<a href="https://colab.research.google.com/github/rg-smith/remote-sensing-hydro-2025/blob/main/lecture12-data-science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 12 - Data Science Methods


First we need to install a couple packages. If this shows an error after running, try the next code block. If it runs without an error, then you should be ok.

In [ ]:
pip install geemap pycrs rasterio

Now we will load necessary packages

In [70]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
from tqdm import tqdm
import zipfile
import os
import pandas as pd
from glob import glob
import geemap
import folium
import branca.colormap as cm
import rasterio
import matplotlib

In [3]:
# you only need to run this once per session
ee.Authenticate()
ee.Initialize(project='replace with project name')

# Define custom functions for working with Google Earth Engine

In [86]:
def add_ee_layer(self, ee_object, name):
    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            range = ee.Image(ee_object).reduceRegion(ee.Reducer.percentile([1, 99]),scale=10000)
            vals = range.getInfo()
            min=list(vals.items())[0][1]
            max=list(vals.items())[1][1]
            vis = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}

            map_id_dict = ee.Image(ee_object).getMapId(vis)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            colormap = cm.LinearColormap(vmin=min,vmax=max,colors=['blue', 'white','red']).to_step(n=10)
            colormap.caption=name
            self.add_child(colormap)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except Exception as e:
        print("Could not display {}".format(name))
        print(e)

# Add EE drawing method to folium (not a function)
folium.Map.add_ee_layer = add_ee_layer

def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function

def gee_zonal_mean_img_coll(imageCollection,geometry,scale=1000):
    reduce_iC = create_reduce_region_function(geometry = geometry, scale=scale)
    stat_fc = ee.FeatureCollection(imageCollection.map(reduce_iC)).filter(ee.Filter.notNull(imageCollection.first().bandNames()))
    fc_dict = fc_to_dict(stat_fc).getInfo()

    df = pd.DataFrame(fc_dict)
    df['date'] = pd.to_datetime(df['millis'],unit='ms')
    return(df)

def gee_zonal_mean(date1,date2,geometry,collection_name,band_name,scale=1000):
     imcol = ee.ImageCollection(collection_name).select(band_name).filterDate(date1,date2)
     df = gee_zonal_mean_img_coll(imcol,geometry,scale=scale)
     return(df)


# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def ee_imgcoll_to_df_point(imagecollection, lat,lon):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    poi = ee.Geometry.Point(lon, lat)
    arr = imagecollection.getRegion(poi,1000).getInfo()

    list_of_bands = imagecollection.first().bandNames().getInfo()

    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

# to get the link to download an earth engine image
def getLink(image,fname,aoi,scale=1000):
  link = image.getDownloadURL({
    'scale': scale,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi,
    'name': fname})
  # print(link)
  return(link)

# create an earth engine geometry polygon
def addGeometry(min_lon,max_lon,min_lat,max_lat):
  geom = ee.Geometry.Polygon(
      [[[min_lon, max_lat],
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat]]])
  return(geom)

def get_imgcollection(date1,date2,geometry,collection_name,band_name,function='mean'):
  collection = ee.ImageCollection(collection_name)
  if function=='mean':
      img = collection.filterDate(date1,date2).select(band_name).mean().clip(geometry)
  if function=='sum':
      img = collection.filterDate(date1,date2).select(band_name).sum().clip(geometry)
 # range = img.reduceRegion(ee.Reducer.percentile([1, 99]),scale=10000)
 # vals = range.getInfo()
 # min=list(vals.items())[0][1]
 # max=list(vals.items())[1][1]
 # visParams = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}
  return(img)

def download_img(img,geom,fname,scale=1000):
    linkname = getLink(img,fname,geom,scale=scale)
    response = requests.get(linkname, stream=True)
    zipped = fname+'.zip'
    with open(zipped, "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)

    with zipfile.ZipFile(zipped, 'r') as zip_ref:
        zip_ref.extractall('')
    os.remove(zipped)


def aggregate_by_water_year(df,date_col,agg_column,agg_fun='sum'):
    df['water_year'] = df[date_col].dt.year.where(df[date_col].dt.month < 10, df[date_col].dt.year + 1)
    df_agg = df.groupby('water_year').agg({agg_column:[agg_fun]})
    return(df_agg)

def download_imgcollection(collection_name,band_name,date1,date2,geometry,dirname,max_num=100,scale=1000):
  count=0
  collection = ee.ImageCollection(collection_name).select(band_name).filterDate(date1,date2)
  list = collection.filterBounds(geometry).toList(collection.size())

  max_num = min(collection.size().getInfo(),max_num)
  print(collection.size().getInfo())
  dates = []
  print('Creating directory '+dirname)
  try:
    os.mkdir(dirname)
    print(f"Directory '{dirname}' created successfully.")
  except FileExistsError:
      print(f"Directory '{dirname}' already exists.")

  for i in range(max_num):
    img = ee.Image(list.get(i)).clip(geometry)

    date_millis = img.get('system:time_start')
    date = ee.Date(date_millis)

    date_string = date.format('YYYY-MM-dd')
    try:
      dates.append(date_string.getInfo())
      print('downloading image number '+str(count))
      download_img(img,geom,dirname+'/'+f"{i:03d}"+'_'+dirname,scale=scale)
      count+=1
    except Exception as e:
      continue
  dates = pd.DataFrame(dates,columns=['date'])
  dates.to_csv(dirname+'/dates.csv')

def load_imgcollection(dirname):
  files = glob(dirname+'/*.tif')
  dates = pd.read_csv(dirname+'/dates.csv',parse_dates=['date'])

  for i in range(len(files)):
    if i==0:
      img = rasterio.open(files[i])
      data = img.read(1)
      array = np.zeros((dates.shape[0],img.shape[0],img.shape[1]))
      array[i,:,:] = data
    else:
      img = rasterio.open(files[i])
      data = img.read(1)
      array[i,:,:] = data
  return(array,dates)

def plot_imgcollection(array,dates):
  nrows = int(np.sqrt(dates.shape[0]))
  ncols = int(np.ceil(dates.shape[0]/nrows))
  print('nrows: '+str(nrows)+', ncols: '+str(ncols))

  fig,ax=plt.subplots(nrows=nrows,ncols=ncols,figsize=(15,15))

  for kk in range(dates.shape[0]):
    row = int(kk/ncols)
    col = kk-ncols*row
    im = ax[row,col].imshow(np.squeeze(array[kk,:,:]))
    ax[row,col].set_title(dates.iloc[kk,1])
    fig.colorbar(im,ax=ax[row,col])

# Download and plot ET data

In [78]:
start='2020-04-01'
end='2023-04-30'

left, bottom, top, right = -104.9, 40.26,40.7,-104.3
geom = addGeometry(left,right,bottom,top)


In [ ]:
download_imgcollection('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0','et_ensemble_mad',
                       start,end,geom,'ET5',max_num=100,scale=100)

In [ ]:
# load and plot all images
array,dates=load_imgcollection('ET5')

plot_imgcollection(array,dates)

# Run PCA

In [ ]:
# run PCA on ET data, and plot components

from sklearn.decomposition import PCA
n_components = 3

array_2d = np.reshape(array,(array.shape[0],array.shape[1]*array.shape[2]))

nansum = np.sum(np.isnan(array_2d),axis=0)
nonan = np.where(nansum==0)[0]

array_2d_nonan = array_2d[:,nonan]

array_2d_transpose = np.transpose(array_2d_nonan)

pca = PCA(n_components=n_components)
pca.fit_transform(array_2d_transpose)

scores = pca.transform(array_2d_transpose)
components = pca.components_

fig,ax=plt.subplots()

for kk in range(n_components):
    ax.plot(dates['date'],components[kk,:],label='Component '+str(kk+1))
ax.legend()
ax.tick_params(axis='x', rotation=45)

In [ ]:
# re-transpose principal component scores, then map them
scores_t = np.transpose(scores)

scores_full = np.zeros((n_components,array_2d.shape[1]))
scores_full[:,:] = np.nan
scores_full[:,nonan] = scores_t

nrows = 1
ncols = 3

fig,ax=plt.subplots(nrows=nrows,ncols=ncols,figsize=(12,5))

for kk in range(n_components):
    scores_2d = scores_full[kk,:].reshape((array.shape[1],array.shape[2]))

    col = kk
    im = ax[col].imshow(scores_2d)
    ax[col].set_title('Component '+str(kk+1))
    fig.colorbar(im,ax=ax[col])

# Plot scores of principal components on a folium map

In [ ]:
my_map = folium.Map(location=[40.5, -105.7], zoom_start=9)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(my_map)

my_map.add_ee_layer(geom,'geom')

for kk in range(n_components):
  scores_2d = scores_full[kk,:].reshape((array.shape[1],array.shape[2]))
  folium.raster_layers.ImageOverlay(
          image=scores_2d,
          bounds=[[bottom, left], [top, right]],
          opacity=1,
          colormap=cm.LinearColormap(['red', 'yellow', 'green'],vmin=np.nanmin(scores_2d),vmax=np.nanmax(scores_2d)),
          name='component '+str(kk+1)
      ).add_to(my_map)

mean_et = np.nanmean(array,axis=0)
folium.raster_layers.ImageOverlay(
          image=mean_et,
          bounds=[[bottom, left], [top, right]],
          opacity=1,
          colormap=cm.LinearColormap(['red', 'yellow', 'green'],vmin=np.nanmin(mean_et),vmax=np.nanmax(mean_et)),
          name='mean ET'
      ).add_to(my_map)

my_map.add_child(folium.LayerControl())
display(my_map)